# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-2.2999e-02,  1.0272e-01,  4.9435e-02,  2.6374e-01, -1.3502e-01,
         -4.0136e-02, -5.6576e-03, -1.3490e-01, -1.0808e-01,  2.1192e-01],
        [ 1.7046e-02, -4.5447e-03,  1.2254e-02,  3.6535e-01, -4.9352e-02,
          1.3135e-04, -8.0115e-02, -4.9918e-02, -1.2148e-01,  2.0024e-01]],
       grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[-0.1069,  0.0134, -0.1163,  0.0059,  0.2715, -0.0306, -0.1391,  0.1152,
          0.0656,  0.1814],
        [-0.0555,  0.0291, -0.0251,  0.0817,  0.2947, -0.0296,  0.0117,  0.1048,
          0.1854,  0.0761]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2094,  0.0216, -0.0705, -0.1138, -0.1646, -0.1235,  0.0429, -0.1646,
          0.1413,  0.1265],
        [-0.1232, -0.1373, -0.0710, -0.1217, -0.1953, -0.1023,  0.1043, -0.1269,
          0.0208, -0.0370]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0509, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2035, grad_fn=<SumBackward0>)